In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [89]:
data = pd.read_csv(r'C:\Users\aa\PyCharmMiscProject\Tiktok_analysis\tiktok_dataset_.csv', sep=';')

In [90]:
data.head()

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296,19425,241,1,0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877,77355,19034,1161,684
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185,97690,2858,833,329
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506,239954,34812,1234,584
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167,34987,4110,547,152


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19382 entries, 0 to 19381
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   #                         19382 non-null  int64  
 1   claim_status              19084 non-null  object 
 2   video_id                  19382 non-null  int64  
 3   video_duration_sec        19382 non-null  int64  
 4   video_transcription_text  19084 non-null  object 
 5   verified_status           19382 non-null  object 
 6   author_ban_status         19382 non-null  object 
 7   video_view_count          19084 non-null  float64
 8   video_like_count          19084 non-null  float64
 9   video_share_count         19084 non-null  float64
 10  video_download_count      19084 non-null  float64
 11  video_comment_count       19084 non-null  float64
dtypes: float64(5), int64(3), object(4)
memory usage: 1.8+ MB


In [92]:
data.describe()

,#,video_id,video_duration_sec,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
count,19084.000000,1.908400e+04,19084.000000,19084.000000,19084.000000,19084.000000,19084.000000,19084.000000
mean,9542.500000,5.624840e+09,32.423811,254708.558688,84304.636030,16735.248323,1049.429627,349.312146
std,5509.220604,2.537030e+09,16.226470,322893.280814,133420.546814,32036.174350,2004.299894,799.638865
min,1.000000,1.234959e+09,5.000000,20.000000,0.000000,0.000000,0.000000,0.000000
25%,4771.750000,3.425100e+09,18.000000,4942.500000,810.750000,115.000000,7.000000,1.000000
50%,9542.500000,5.609500e+09,32.000000,9954.500000,3403.500000,717.000000,46.000000,9.000000
75%,14313.250000,7.840823e+09,47.000000,504327.000000,125020.000000,18222.000000,1156.250000,292.000000
max,19084.000000,9.999873e+09,60.000000,999817.000000,657830.000000,256130.000000,14994.000000,9599.000000


In [93]:
data.shape

(19084, 12)

In [94]:
data.isnull().sum()

#                           0
claim_status                0
video_id                    0
video_duration_sec          0
video_transcription_text    0
verified_status             0
author_ban_status           0
video_view_count            0
video_like_count            0
video_share_count           0
video_download_count        0
video_comment_count         0
dtype: int64

In [95]:
data['verified_status'].value_counts()

verified_status
not verified    17884
verified         1200
Name: count, dtype: int64

In [96]:
data['claim_status'].value_counts()

claim_status
claim      9608
opinion    9476
Name: count, dtype: int64

Data Preprocessing

	Identify missing values (NaN) in the dataset.

	Decide on a strategy to handle them (e.g., filling with mean/median/mode, forward/backward filling, or dropping rows/columns).


In [98]:
print(data['claim_status'].isnull())

0        False
1        False
2        False
3        False
4        False
         ...  
19079    False
19080    False
19081    False
19082    False
19083    False
Name: claim_status, Length: 19084, dtype: bool


In [97]:
data['claim_status'].value_counts(dropna=False)

claim_status
claim      9608
opinion    9476
Name: count, dtype: int64

In [103]:
missing_percentage = (data.isnull().sum() / len(data)) * 100
print(missing_percentage)

#                           0.0
claim_status                0.0
video_id                    0.0
video_duration_sec          0.0
video_transcription_text    0.0
verified_status             0.0
author_ban_status           0.0
video_view_count            0.0
video_like_count            0.0
video_share_count           0.0
video_download_count        0.0
video_comment_count         0.0
dtype: float64


In [100]:
null_values = data.isnull()
print(null_values)

           #  claim_status  video_id  video_duration_sec  \
0      False         False     False               False   
1      False         False     False               False   
2      False         False     False               False   
3      False         False     False               False   
4      False         False     False               False   
...      ...           ...       ...                 ...   
19079  False         False     False               False   
19080  False         False     False               False   
19081  False         False     False               False   
19082  False         False     False               False   
19083  False         False     False               False   

       video_transcription_text  verified_status  author_ban_status  \
0                         False            False              False   
1                         False            False              False   
2                         False            False              Fals

In [59]:
null_summary = data.isnull().sum().reset_index()
null_summary.columns = ['Column', 'MissingValues']
null_summary['Percentage'] = (null_summary['MissingValues'] / len(data)) * 100
print(null_summary)

                      Column  MissingValues  Percentage
0                          #              0    0.000000
1               claim_status            298    1.537509
2                   video_id              0    0.000000
3         video_duration_sec              0    0.000000
4   video_transcription_text            298    1.537509
5            verified_status              0    0.000000
6          author_ban_status              0    0.000000
7           video_view_count            298    1.537509
8           video_like_count            298    1.537509
9          video_share_count            298    1.537509
10      video_download_count            298    1.537509
11       video_comment_count            298    1.537509


In [60]:
only_nulls = data[data.isnull()]
print(only_nulls)


        # claim_status  video_id  video_duration_sec video_transcription_text  \
0     NaN          NaN       NaN                 NaN                      NaN   
1     NaN          NaN       NaN                 NaN                      NaN   
2     NaN          NaN       NaN                 NaN                      NaN   
3     NaN          NaN       NaN                 NaN                      NaN   
4     NaN          NaN       NaN                 NaN                      NaN   
...    ..          ...       ...                 ...                      ...   
19377 NaN          NaN       NaN                 NaN                      NaN   
19378 NaN          NaN       NaN                 NaN                      NaN   
19379 NaN          NaN       NaN                 NaN                      NaN   
19380 NaN          NaN       NaN                 NaN                      NaN   
19381 NaN          NaN       NaN                 NaN                      NaN   

      verified_status autho

Total missing values by column

Rows with missing values

Percentage of missing values by column




In [61]:
print("Total missing values by column:")
print(data.isnull().sum())

print("\nRows with missing values:")
print(data[data.isnull().any(axis=1)])

print("\nPercentage of missing values by column:")
print((data.isnull().sum() / len(data)) * 100)

Total missing values by column:
#                             0
claim_status                298
video_id                      0
video_duration_sec            0
video_transcription_text    298
verified_status               0
author_ban_status             0
video_view_count            298
video_like_count            298
video_share_count           298
video_download_count        298
video_comment_count         298
dtype: int64

Rows with missing values:
           # claim_status    video_id  video_duration_sec  \
19084  19085          NaN  4380513697                  39   
19085  19086          NaN  8352130892                  60   
19086  19087          NaN  4443076562                  25   
19087  19088          NaN  8328300333                   7   
19088  19089          NaN  3968729520                   8   
...      ...          ...         ...                 ...   
19377  19378          NaN  7578226840                  21   
19378  19379          NaN  6079236179                  5

In [62]:
# Drop all rows with null values
data = data.dropna()
print("Shape of the DataFrame after dropping rows with null values:", data.shape)

rows_with_null = data[data.isnull().any(axis=1)]
print(rows_with_null)

Shape of the DataFrame after dropping rows with null values: (19084, 12)
Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [63]:
data.isnull().sum()

#                           0
claim_status                0
video_id                    0
video_duration_sec          0
video_transcription_text    0
verified_status             0
author_ban_status           0
video_view_count            0
video_like_count            0
video_share_count           0
video_download_count        0
video_comment_count         0
dtype: int64

	Check for and remove any duplicate rows to ensure that each entry is unique.

In [64]:
data.nunique()

#                           19084
claim_status                    2
video_id                    19084
video_duration_sec             56
video_transcription_text    19012
verified_status                 2
author_ban_status               3
video_view_count            15632
video_like_count            12224
video_share_count            9231
video_download_count         4336
video_comment_count          2424
dtype: int64

In [65]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
19079    False
19080    False
19081    False
19082    False
19083    False
Length: 19084, dtype: bool

In [66]:
duplicates = data[data.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


	Identify and handle outliers in numerical columns like views, likes, comments, etc., using methods such as the IQR method or Z-score.


In [67]:
# Calculate the IQR for numerical columns
numerical_columns = ['video_view_count', 'video_like_count', 'video_share_count', 'video_download_count']
Q1 = data[numerical_columns].quantile(0.25)
Q3 = data[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("Shape before removing outliers:", data.shape)
data = data[~((data[numerical_columns] < lower_bound) | (data[numerical_columns] > upper_bound)).any(axis=1)]
print("Shape after removing outliers:", data.shape)

Shape before removing outliers: (19084, 12)
Shape after removing outliers: (15437, 12)


	Convert categorical variables like verified_status, author_ban_status, etc., into binary or dummy variables if needed for modeling.

In [68]:
data.head()

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0


In [69]:
categorical_columns = data.select_dtypes(include=['object','category']).columns
print("________________Categorical columns________________________")
print(categorical_columns)

________________Categorical columns________________________
Index(['claim_status', 'video_transcription_text', 'verified_status',
       'author_ban_status'],
      dtype='object')


In [70]:
data.columns

Index(['#', 'claim_status', 'video_id', 'video_duration_sec',
       'video_transcription_text', 'verified_status', 'author_ban_status',
       'video_view_count', 'video_like_count', 'video_share_count',
       'video_download_count', 'video_comment_count'],
      dtype='object')

In [71]:
duplicates = data[data.duplicated()]
print("Sürüşən sətirlər:")
print(duplicates)

Sürüşən sətirlər:
Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [72]:
# Sütunların sürüşən olub-olmadığını yoxlamaq
duplicate_columns = data.T.duplicated()
duplicate_columns_names = data.columns[duplicate_columns]

print("Sürüşən sütunlar:")
print(duplicate_columns_names)


Sürüşən sütunlar:
Index([], dtype='object')


In [73]:
duplicates = data[data.duplicated(keep=False)]
print(duplicates)


Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [74]:
duplicates = data[data.duplicated(subset=['claim_status', 'video_id', 'video_duration_sec'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [75]:
data_trimmed = data.apply(lambda x: x.astype(str).str.strip().str.lower() if x.dtype == "object" else x)
duplicates = data_trimmed[data_trimmed.duplicated(keep=False)]
print(duplicates)


Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [76]:
duplicate_columns = data.T.duplicated(keep=False)
print("Sürüşən sütunlar:", data.columns[duplicate_columns].tolist())

Sürüşən sütunlar: []


In [77]:
for col1 in data.columns:
    for col2 in data.columns:
        if col1 != col2 and data[col1].equals(data[col2]):
            print(f"Sütunlar eynidir: {col1} və {col2}")

In [78]:
print(data.nunique())


#                           15437
claim_status                    2
video_id                    15437
video_duration_sec             56
video_transcription_text    15412
verified_status                 2
author_ban_status               3
video_view_count            12007
video_like_count             8601
video_share_count            5761
video_download_count         2091
video_comment_count          1102
dtype: int64


In [79]:
columns_to_check = ['claim_status', 'video_transcription_text', 'verified_status', 'author_ban_status']
duplicates = data[data.duplicated(subset=columns_to_check, keep=False)]
print(duplicates)


         # claim_status    video_id  video_duration_sec  \
2502  2503        claim  2307103802                  36   
2505  2506        claim  6457026547                  30   
2510  2511        claim  2425568026                  15   
2548  2549        claim  6553958541                  12   
2560  2561        claim  7934238615                  18   
2943  2944        claim  7402304136                  15   
2945  2946        claim  5602633649                  58   
2956  2957        claim  5473025809                  16   
3002  3003        claim  2338854116                  34   
3008  3009        claim  7820590426                  51   
3217  3218        claim  9318941735                  33   
3229  3230        claim  6953288910                  55   
3665  3666        claim  5551035336                  20   
3683  3684        claim  9058571671                  20   
3892  3893        claim  3314084381                  34   
3940  3941        claim  9195625646                  34 

In [80]:
columns_to_check = ['video_view_count', 'video_like_count', 'video_share_count']
duplicates = data[data.duplicated(subset=columns_to_check, keep=False)]
print(duplicates)


           # claim_status    video_id  video_duration_sec  \
15881  15882      opinion  1341587679                  55   
16136  16137      opinion  2701435506                  54   

                                video_transcription_text verified_status  \
15881  my colleagues' point of view is that a giant t...    not verified   
16136  my colleagues' thinking is that it would take ...    not verified   

      author_ban_status  video_view_count  video_like_count  \
15881            active              51.0              12.0   
16136            active              51.0              12.0   

       video_share_count  video_download_count  video_comment_count  
15881                3.0                   0.0                  0.0  
16136                3.0                   0.0                  0.0  
